<a href="https://colab.research.google.com/github/offani/Social-Media-Topic-Modelling/blob/main/DC_Bert_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install bertopic sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
#!/usr/bin/env python

import pandas as pd
import re
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# -------------------------------
# Preprocessing Function
# -------------------------------
def preprocess_text(text):
    """
    Clean text by removing URLs, mentions, hashtags, punctuation, numbers,
    and converting to lowercase.
    """
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user references and hashtags
    text = re.sub(r'\@\w+|\#', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Convert to lowercase
    return text.lower()

# -------------------------------
# CSV Reader with Chunking
# -------------------------------
def read_csv_and_process(file_path, chunksize=10000, text_column='text'):
    """
    Read a large CSV file in chunks, preprocess text data, and return a list of processed texts.
    """
    texts = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        if text_column in chunk.columns:
            # Ensure all entries are strings and preprocess them
            processed = chunk[text_column].astype(str).apply(preprocess_text).tolist()
            texts.extend(processed)
    return texts

# -------------------------------
# Main Function
# -------------------------------
def main():
    # Set the path to your CSV file and the name of the text column
    file_path = '/content/sentimentdataset.csv'  # <-- Replace with your CSV file path
    text_column = 'Text'  # <-- Replace with your CSV column name containing social media texts

    print("Reading and processing CSV file...")
    texts = read_csv_and_process(file_path, chunksize=10000, text_column=text_column)
    print(f"Total texts processed: {len(texts)}")

    print("Generating embeddings...")
    # Initialize the SentenceTransformer model for embeddings
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embedding_model.encode(texts, show_progress_bar=True)

    print("Fitting BERTopic model...")
    # Initialize BERTopic with the embedding model
    topic_model = BERTopic(embedding_model=embedding_model)
    topics, probabilities = topic_model.fit_transform(texts, embeddings)

    print("Topic Information:")
    topic_info = topic_model.get_topic_info()
    print(topic_info.head())

    # Visualize topics and save the interactive plot as HTML
    fig = topic_model.visualize_topics()
    fig.write_html("topics.html")
    print("Topics visualization saved as topics.html")

    # Save the trained BERTopic model for future use
    topic_model.save("bertopic_model")
    print("BERTopic model saved as bertopic_model")

if __name__ == "__main__":
    main()


Reading and processing CSV file...
Total texts processed: 732
Generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Fitting BERTopic model...
Topic Information:
   Topic  Count                                     Name  \
0     -1    147                          -1_the_on_of_to   
1      0    100                    0_of_the_shattered_in   
2      1     69                 1_the_of_serenity_beauty   
3      2     46  2_gratitude_kindness_compassion_empathy   
4      3     36   3_excitement_surprise_friends_birthday   

                                      Representation  \
0  [the, on, of, to, and, in, new, for, with, pro...   
1  [of, the, shattered, in, despair, by, emotions...   
2  [the, of, serenity, beauty, sunset, garden, in...   
3  [gratitude, kindness, compassion, empathy, wit...   
4  [excitement, surprise, friends, birthday, week...   

                                 Representative_Docs  
0  [savoring the warmth of a cup of cocoa on a ch...  
1  [ shattered dreams lie on the floor like fragm...  
2  [in the garden of contentment each bloom whisp...  
3  [ a compassionate rain tears of em

2025-03-17 17:30:23,313 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Topics visualization saved as topics.html
BERTopic model saved as bertopic_model


##actual data

In [5]:
pip  install scikit-learn

In [8]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups


df=pd.read_csv('/content/sentimentdataset.csv')
df=df["Text"]
# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data'][:2000]
docs=df.tolist()

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)


In [9]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,175,-1_the_of_in_to,"[the, of, in, to, and, with, on, day, for, my]","[ Proudly scaling the peaks of achievement, a ..."
1,0,78,0_art_project_creativity_new,"[art, project, creativity, new, on, in, the, a...",[ Coding a new project with enthusiasm. ...
2,1,53,1_excitement_surprise_weekend_friends,"[excitement, surprise, weekend, friends, good,...",[ Arousal of excitement for an upcoming advent...
3,2,50,2_shattered_despair_of_in,"[shattered, despair, of, in, grief, the, like,...",[ Shattered dreams lie on the floor like fragm...
4,3,41,3_concert_music_dance_the,"[concert, music, dance, the, of, at, symphony,...","[Attended a classical music concert, feeling t..."
5,4,37,4_kindness_gratitude_compassion_empathy,"[kindness, gratitude, compassion, empathy, cha...","[ A compassionate rain, tears of empathy falli..."
6,5,34,5_sunset_of_the_garden,"[sunset, of, the, garden, beauty, vastness, fl...",[A sense of wonder at the vastness of the cosm...
7,6,29,6_ancient_curiosity_of_hidden,"[ancient, curiosity, of, hidden, knowledge, an...",[Exploring the historical architecture of an a...
8,7,29,7_bitter_resentment_jealousy_bitterness,"[bitter, resentment, jealousy, bitterness, onl...","[ Bitterness, a bitter aftertaste lingering on..."
9,8,27,8_personal_fitness_workout_achieving,"[personal, fitness, workout, achieving, challe...",[Pride in achieving a personal milestone in ca...


In [10]:
 topic_model.get_topic(0)

[('art', np.float64(0.04757835147969114)),
 ('project', np.float64(0.04262659855156514)),
 ('creativity', np.float64(0.03342002630383551)),
 ('new', np.float64(0.032757714780632774)),
 ('on', np.float64(0.030918554999681505)),
 ('in', np.float64(0.028519670777588574)),
 ('the', np.float64(0.028452848130312156)),
 ('and', np.float64(0.028141877652811745)),
 ('to', np.float64(0.02713190800292246)),
 ('is', np.float64(0.025977613243650027))]

In [11]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Enjoying a beautiful day at the park! ...,1,1_excitement_surprise_weekend_friends,"[excitement, surprise, weekend, friends, good,...",[ Arousal of excitement for an upcoming advent...,excitement - surprise - weekend - friends - go...,0.608621,False
1,Traffic was terrible this morning. ...,-1,-1_the_of_in_to,"[the, of, in, to, and, with, on, day, for, my]","[ Proudly scaling the peaks of achievement, a ...",the - of - in - to - and - with - on - day - f...,0.000000,False
2,Just finished an amazing workout! 💪 ...,8,8_personal_fitness_workout_achieving,"[personal, fitness, workout, achieving, challe...",[Pride in achieving a personal milestone in ca...,personal - fitness - workout - achieving - cha...,1.000000,False
3,Excited about the upcoming weekend getaway! ...,1,1_excitement_surprise_weekend_friends,"[excitement, surprise, weekend, friends, good,...",[ Arousal of excitement for an upcoming advent...,excitement - surprise - weekend - friends - go...,0.955882,False
4,Trying out a new recipe for dinner tonight. ...,13,13_recipe_flavors_savoring_kitchen,"[recipe, flavors, savoring, kitchen, meal, cul...","[Embarking on a culinary odyssey, savoring fla...",recipe - flavors - savoring - kitchen - meal -...,1.000000,False
...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,0,0_art_project_creativity_new,"[art, project, creativity, new, on, in, the, a...",[ Coding a new project with enthusiasm. ...,art - project - creativity - new - on - in - t...,1.000000,False
728,Attending a surprise birthday party organized ...,1,1_excitement_surprise_weekend_friends,"[excitement, surprise, weekend, friends, good,...",[ Arousal of excitement for an upcoming advent...,excitement - surprise - weekend - friends - go...,0.483726,False
729,Successfully fundraising for a school charity ...,4,4_kindness_gratitude_compassion_empathy,"[kindness, gratitude, compassion, empathy, cha...","[ A compassionate rain, tears of empathy falli...",kindness - gratitude - compassion - empathy - ...,0.601818,False
730,"Participating in a multicultural festival, cel...",1,1_excitement_surprise_weekend_friends,"[excitement, surprise, weekend, friends, good,...",[ Arousal of excitement for an upcoming advent...,excitement - surprise - weekend - friends - go...,0.360877,False


In [14]:
from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

topics, probs = topic_model.fit_transform(docs)

In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,178,-1_acceptance_experiencing_life_beauty,"[acceptance, experiencing, life, beauty, journ...","[ Embracing the ebb and flow of life, finding ..."
1,0,57,0_creativity_artistic_art_exhibition,"[creativity, artistic, art, exhibition, explor...","[Joined a nature photography club, capturing t..."
2,1,55,1_excitement_exhilarating_thrilling_celebrating,"[excitement, exhilarating, thrilling, celebrat...",[Exploring a new hiking trail with friends ove...
3,2,53,2_shattered_hopelessness_dreams_despair,"[shattered, hopelessness, dreams, despair, tea...",[ Shattered dreams lie on the floor like fragm...
4,3,44,3_dancing_dance_concert_ballroom,"[dancing, dance, concert, ballroom, melody, mu...","[ Confident strides in the dance of life, a ba..."
5,4,39,4_empathy_compassion_compassionate_kindness,"[empathy, compassion, compassionate, kindness,...","[ A compassionate rain, tears of empathy falli..."
6,5,36,5_achievement_accomplishment_achieving_milestone,"[achievement, accomplishment, achieving, miles...","[ Proudly scaling the peaks of achievement, a ..."
7,6,27,6_embraced_optimism_blooms_gardener,"[embraced, optimism, blooms, gardener, dreams,...","[ Embraced by the hopeful dawn, a gardener sow..."
8,7,27,7_archaeological_ancient_exploration_mysteries,"[archaeological, ancient, exploration, mysteri...",[Exploring the historical architecture of an a...
9,8,27,8_bitterness_bitter_resentment_anger,"[bitterness, bitter, resentment, anger, discon...","[ Bitterness, a bitter aftertaste lingering on..."


from matplotlib import pyplot as plt
_df_0['Topic'].plot(kind='hist', bins=20, title='Topic')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Count'].plot(kind='hist', bins=20, title='Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='Topic', y='Count', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Topic']
  ys = series['Count']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('Topic', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Topic')
_ = plt.ylabel('Count')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Topic']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Topic'}, axis=1)
              .sort_values('Topic', ascending=True))
  xs = counted['Topic']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('Topic', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Topic')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['Topic'].plot(kind='line', figsize=(8, 4), title='Topic')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['Count'].plot(kind='line', figsize=(8, 4), title='Count')
plt.gca().spines[['top', 'right']].set_visible(False)

In [17]:
 topic_model.get_topic(0)

[('creativity', np.float32(0.53871036)),
 ('artistic', np.float32(0.5207176)),
 ('art', np.float32(0.50224495)),
 ('exhibition', np.float32(0.48173124)),
 ('exploring', np.float32(0.4724882)),
 ('painting', np.float32(0.4674537)),
 ('photography', np.float32(0.46608478)),
 ('mural', np.float32(0.45253563)),
 ('artists', np.float32(0.45224893)),
 ('seniorphotographer', np.float32(0.43745244))]

In [18]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Enjoying a beautiful day at the park! ...,1,1_excitement_exhilarating_thrilling_celebrating,"[excitement, exhilarating, thrilling, celebrat...",[Exploring a new hiking trail with friends ove...,excitement - exhilarating - thrilling - celebr...,0.656644,False
1,Traffic was terrible this morning. ...,-1,-1_acceptance_experiencing_life_beauty,"[acceptance, experiencing, life, beauty, journ...","[ Embracing the ebb and flow of life, finding ...",acceptance - experiencing - life - beauty - jo...,0.000000,False
2,Just finished an amazing workout! 💪 ...,5,5_achievement_accomplishment_achieving_milestone,"[achievement, accomplishment, achieving, miles...","[ Proudly scaling the peaks of achievement, a ...",achievement - accomplishment - achieving - mil...,1.000000,False
3,Excited about the upcoming weekend getaway! ...,1,1_excitement_exhilarating_thrilling_celebrating,"[excitement, exhilarating, thrilling, celebrat...",[Exploring a new hiking trail with friends ove...,excitement - exhilarating - thrilling - celebr...,0.887980,False
4,Trying out a new recipe for dinner tonight. ...,14,14_culinary_recipes_recipe_cooking,"[culinary, recipes, recipe, cooking, chef, coo...","[Embarking on a culinary odyssey, savoring fla...",culinary - recipes - recipe - cooking - chef -...,1.000000,False
...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,0,0_creativity_artistic_art_exhibition,"[creativity, artistic, art, exhibition, explor...","[Joined a nature photography club, capturing t...",creativity - artistic - art - exhibition - exp...,1.000000,False
728,Attending a surprise birthday party organized ...,1,1_excitement_exhilarating_thrilling_celebrating,"[excitement, exhilarating, thrilling, celebrat...",[Exploring a new hiking trail with friends ove...,excitement - exhilarating - thrilling - celebr...,0.632764,False
729,Successfully fundraising for a school charity ...,4,4_empathy_compassion_compassionate_kindness,"[empathy, compassion, compassionate, kindness,...","[ A compassionate rain, tears of empathy falli...",empathy - compassion - compassionate - kindnes...,0.708838,False
730,"Participating in a multicultural festival, cel...",-1,-1_acceptance_experiencing_life_beauty,"[acceptance, experiencing, life, beauty, journ...","[ Embracing the ebb and flow of life, finding ...",acceptance - experiencing - life - beauty - jo...,0.000000,False


##docs moifications

In [4]:
pip install hdbscan sentence_transformers scikit-learn bertopic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [1]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer


# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model # Step 6 - (Optional) Fine-tune topic representations
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import pandas as pd
df=pd.read_csv('/content/sentimentdataset.csv')
df=df["Text"]
docs=df
topics, probs = topic_model.fit_transform(docs)

In [5]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,94,-1_imperfections_acceptance_self_embracing,"[imperfections, acceptance, self, embracing, c...","[ Embracing the beauty in imperfections, findi..."
1,0,180,0_solitude_echoes_loneliness_solitary,"[solitude, echoes, loneliness, solitary, silen...","[The echoes of solitude, a silent conversation..."
2,1,151,1_creativity_art_exhibition_painting,"[creativity, art, exhibition, painting, photog...","[Participated in a community art class, unleas..."
3,2,61,2_serene_meditation_serenity_tranquility,"[serene, meditation, serenity, tranquility, pe...","[In the serene beauty of a sunset, nature unfo..."
4,3,54,3_achievement_accomplishment_achieving_milestone,"[achievement, accomplishment, achieving, miles...","[ Proudly scaling the peaks of achievement, a ..."
5,4,45,4_excitement_celebrating_celebration_exhilarating,"[excitement, celebrating, celebration, exhilar...",[ Arousal of excitement for an upcoming advent...
6,5,43,5_concert_music_concertvibes_seniormusic,"[concert, music, concertvibes, seniormusic, se...",[ Attending a live music concert tonight. ...
7,6,37,6_empathy_compassion_compassionate_kindness,"[empathy, compassion, compassionate, kindness,...","[ A compassionate rain, tears of empathy falli..."
8,7,26,7_optimism_optimistic_hopeful_sunrise,"[optimism, optimistic, hopeful, sunrise, embra...","[ Embraced by the hopeful dawn, a gardener sow..."
9,8,23,8_carnival_amusement_playful_carousel,"[carnival, amusement, playful, carousel, circu...","[ A playful escapade in the carnival of life, ..."


In [6]:
 topic_model.get_topic(0)

[('solitude', np.float32(0.5681821)),
 ('echoes', np.float32(0.53003454)),
 ('loneliness', np.float32(0.5026607)),
 ('solitary', np.float32(0.4977274)),
 ('silence', np.float32(0.49484357)),
 ('despair', np.float32(0.4576461)),
 ('silent', np.float32(0.45245162)),
 ('whispers', np.float32(0.42949918)),
 ('betrayal', np.float32(0.42166364)),
 ('labyrinth', np.float32(0.40566495))]

In [7]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Enjoying a beautiful day at the park! ...,4,4_excitement_celebrating_celebration_exhilarating,"[excitement, celebrating, celebration, exhilar...",[ Arousal of excitement for an upcoming advent...,excitement - celebrating - celebration - exhil...,0.937216,False
1,Traffic was terrible this morning. ...,0,0_solitude_echoes_loneliness_solitary,"[solitude, echoes, loneliness, solitary, silen...","[The echoes of solitude, a silent conversation...",solitude - echoes - loneliness - solitary - si...,0.800771,False
2,Just finished an amazing workout! 💪 ...,3,3_achievement_accomplishment_achieving_milestone,"[achievement, accomplishment, achieving, miles...","[ Proudly scaling the peaks of achievement, a ...",achievement - accomplishment - achieving - mil...,1.000000,False
3,Excited about the upcoming weekend getaway! ...,4,4_excitement_celebrating_celebration_exhilarating,"[excitement, celebrating, celebration, exhilar...",[ Arousal of excitement for an upcoming advent...,excitement - celebrating - celebration - exhil...,1.000000,False
4,Trying out a new recipe for dinner tonight. ...,9,9_culinary_recipes_cooking_savoring,"[culinary, recipes, cooking, savoring, recipe,...","[Embarking on a culinary adventure, savoring e...",culinary - recipes - cooking - savoring - reci...,1.000000,False
...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,1,1_creativity_art_exhibition_painting,"[creativity, art, exhibition, painting, photog...","[Participated in a community art class, unleas...",creativity - art - exhibition - painting - pho...,1.000000,False
728,Attending a surprise birthday party organized ...,4,4_excitement_celebrating_celebration_exhilarating,"[excitement, celebrating, celebration, exhilar...",[ Arousal of excitement for an upcoming advent...,excitement - celebrating - celebration - exhil...,0.626697,False
729,Successfully fundraising for a school charity ...,6,6_empathy_compassion_compassionate_kindness,"[empathy, compassion, compassionate, kindness,...","[ A compassionate rain, tears of empathy falli...",empathy - compassion - compassionate - kindnes...,0.802790,False
730,"Participating in a multicultural festival, cel...",4,4_excitement_celebrating_celebration_exhilarating,"[excitement, celebrating, celebration, exhilar...",[ Arousal of excitement for an upcoming advent...,excitement - celebrating - celebration - exhil...,0.599697,False
